In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *

In [3]:
fundr = pd.read_csv(f'{DATA_FOLDER}/fund_ratios_1970-2020.csv', index_col=0)
ibes = pd.read_csv(f'{DATA_FOLDER}/ibes_1976-2020.csv', index_col=0)
comp = pd.read_csv(f'{DATA_FOLDER}/comp_1959-2019.csv', index_col=0)

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
fund_r = fundr[fundr['qdate'].notna()]
fund_r['date'] = pd.to_datetime((fund_r['qdate']))
fund_r['jdate']=fund_r['date']+MonthEnd(0)
fund_r['year'] = fund_r['jdate'].dt.year

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [5]:
compvars = [ 'at',
                     'ni',
                     'epspx',
                     'revt',
                     'capx',
                     'naics',
                     'ib',
                     'ebitda',
                     'ebit',
                     'nopi',
                     'pi',
                     'dvt',
                     'be',]

In [6]:
%%time

mdf = comp
mdf = mdf.sort_values(['permno', 'year'], ascending=[True, False])

eng_yr = 5
pred_var_yr = 5
pred_i = list(range(-pred_var_yr, 0))

for i in range(-pred_var_yr, eng_yr+1):
    mdf[f'ni_{i}'] = mdf.groupby('permno')['ni'].shift(i)
    print(i)
    if i in pred_i:
        for var in compvars:
            mdf[f'{var}_{i}'] = mdf.groupby('permno')[var].shift(i)

-5
-4
-3
-2
-1
0
1
2
3
4
5
CPU times: user 10.6 s, sys: 9.73 s, total: 20.4 s
Wall time: 21.2 s


In [7]:
for i in range(1, 6):
    print(i)
    for var in compvars: 
        mdf[f'{var}_yoy{i}'] = ((mdf[f'{var}'] - mdf[f'{var}_-{i}']) / mdf[f'{var}'])**(1/i)

1
2
3
4
5


In [8]:
mdf['jdate'] = pd.to_datetime((mdf['jdate']), format='%Y-%m-%d')
mdf = mdf.merge(fund_r, how='left', on=['permno', 'jdate'])

In [9]:
mdf['year'] = mdf['year_y']
mdf = mdf.drop(columns=['year_x', 'year_y'])

In [10]:
mdf.tail(3)

,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk,ajex,fyear,apdedate,pdate,fdate,cnum,cik,sic2,sic,naics,act,ceq,che,dltt,gdwl,intan,lct,lt,ppegt,ppent,txdb,wcap,dp,dvt,ebit,ebitda,epsfx,epspx,gp,ib,ni,niadj,nopi,opincar,opiti,pi,revt,capx,aqc,dv,chech,dpc,wcapch,ps_x,be,count,permno,jdate,ni_-5,at_-5,epspx_-5,revt_-5,capx_-5,naics_-5,ib_-5,ebitda_-5,ebit_-5,nopi_-5,pi_-5,dvt_-5,be_-5,ni_-4,at_-4,epspx_-4,revt_-4,capx_-4,naics_-4,ib_-4,ebitda_-4,ebit_-4,nopi_-4,pi_-4,dvt_-4,be_-4,ni_-3,at_-3,epspx_-3,revt_-3,capx_-3,naics_-3,ib_-3,ebitda_-3,ebit_-3,nopi_-3,pi_-3,dvt_-3,be_-3,ni_-2,at_-2,epspx_-2,revt_-2,capx_-2,naics_-2,ib_-2,ebitda_-2,ebit_-2,nopi_-2,pi_-2,dvt_-2,be_-2,ni_-1,at_-1,epspx_-1,revt_-1,capx_-1,naics_-1,ib_-1,ebitda_-1,ebit_-1,nopi_-1,pi_-1,dvt_-1,be_-1,ni_0,ni_1,ni_2,ni_3,ni_4,ni_5,at_yoy1,ni_yoy1,epspx_yoy1,revt_yoy1,capx_yoy1,naics_yoy1,ib_yoy1,ebitda_yoy1,ebit_yoy1,nopi_yoy1,pi_yoy1,dvt_yoy1,be_yoy1,at_yoy2,ni_yoy2,epspx_yoy2,revt_yoy2,capx_yoy2,naics_yoy2,ib_yoy2,ebitda_yoy2,ebit_yoy2,nopi_yoy2,pi_yoy2,dvt_yoy2,be_yoy2,at_yoy3,ni_yoy3,epspx_yoy3,revt_yoy3,capx_yoy3,naics_yoy3,ib_yoy3,ebitda_yoy3,ebit_yoy3,nopi_yoy3,pi_yoy3,dvt_yoy3,be_yoy3,at_yoy4,ni_yoy4,epspx_yoy4,revt_yoy4,capx_yoy4,naics_yoy4,ib_yoy4,ebitda_yoy4,ebit_yoy4,nopi_yoy4,pi_yoy4,dvt_yoy4,be_yoy4,at_yoy5,ni_yoy5,epspx_yoy5,revt_yoy5,capx_yoy5,naics_yoy5,ib_yoy5,ebitda_yoy5,ebit_yoy5,nopi_yoy5,pi_yoy5,dvt_yoy5,be_yoy5,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps_y,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,GProf,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward,TICKER,date,year
690031,184996,2009-12-31,130.424,442.151,0.0,442.151,65.702,319.225,5.0,2009.0,2009-12-31,2010-06-16,2010-06-30,88160R101,1318605.0,3711,3711,336111.0,100.559,-253.523,69.627,0.8,0.0,0.0,57.489,64.722,36.265,23.535,0.0,43.07,6.94,0.0,-51.897,-44.957,-0.6,-0.6,16.475,-55.74,-55.74,-55.74,-2.754,NaN,NaN,-55.714,111.943,11.884,0.0,0.0,60.35,6.94,NaN,442.151,NaN,1,93436,2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-55.74,-154.328,-254.411,-396.213,-74.014,-294.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009/12/31,2010/06/30,2010/08/31,NaN,NaN,-14.272,-18.731,-18.731,-18.731,-18.731,16.020,-20.541,NaN,-0.855,-0.671,-0.746,0.248,-0.854,-0.78,-0.552,NaN,0.338,NaN,0.364,-2.759,-1.835,-1.253,-0.672,0.199,-4.375,0.402,0.407,0.402,0.029,0.028,0.578,0.279,0.055,0.185,-0.343,0.011,0.6,0.251,-1.232,-1.432,-1.249,NaN,3.694,NaN,0.728,0.544,2.682,-130.47,-114.76,0.964,1.133,1.572,49.057,3.144,0.802,21.402,15.549,1.765,2.954,3.208,0.336,0.015,0.0,0.06,NaN,NaN,NaN,NaN,NaN,TSLA,2010-06-30,2010.0
690032,184996,2009-12-31,130.424,442.151,0.0,442.151,65.702,319.225,5.0,2009.0,2009-12-31,2010-06-16,2010-06-30,88160R101,1318605.0,3711,3711,336111.0,100.559,-253.523,69.627,0.8,0.0,0.0,57.489,64.722,36.265,23.535,0.0,43.07,6.94,0.0,-51.897,-44.957,-0.6,-0.6,16.475,-55.74,-55.74,-55.74,-2.754,NaN,NaN,-55.714,111.943,11.884,0.0,0.0,60.35,6.94,NaN,442.151,NaN,1,93436,2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [11]:
crsp = pd.read_csv(f'{DATA_FOLDER}/crsp_1959-2019.csv', index_col=0)

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
crsp['jdate'] = pd.to_datetime(crsp['jdate'])
mdf1 = mdf.merge(crsp, on=['jdate', 'permno'], how='left')

In [13]:
ibes['jdate'] = pd.to_datetime(ibes['jdate'])

In [14]:
mdf2 = mdf1.merge(ibes, on=['jdate', 'permno', 'ticker'], how='left')

In [15]:
vars_keep = ['ticker',
             'exchcd',
             'permno',
             'jdate',
               'count',
             'shrout',
             'sic2',
             'sic',
              'siccd',
             'naics',
             'act',
             'ceq',
             'che',
             'dltt',
             'gdwl',
             'intan',
             'lct',
             'lt',
             'ppegt',
             'ppent',
             'txdb',
             'wcap',
             'dp',
             'dvt',
             'ebit',
             'ebitda',
             'epsfx',
             'epspx',
             'gp',
             'ib',
             'ni',
             'niadj',
             'nopi',
             'opincar',
             'opiti',
             'pi',
             'revt',
             'capx',
             'aqc',
             'dv',
             'chech',
             'dpc',
             'wcapch',
             'ps_x',
             'be',
             'ni_-5',
             'at_-5',
             'epspx_-5',
             'revt_-5',
             'capx_-5',
             'naics_-5',
             'ib_-5',
             'ebitda_-5',
             'ebit_-5',
             'nopi_-5',
             'pi_-5',
             'dvt_-5',
             'be_-5',
             'ni_-4',
             'at_-4',
             'epspx_-4',
             'revt_-4',
             'capx_-4',
             'naics_-4',
             'ib_-4',
             'ebitda_-4',
             'ebit_-4',
             'nopi_-4',
             'pi_-4',
             'dvt_-4',
             'be_-4',
             'ni_-3',
             'at_-3',
             'epspx_-3',
             'revt_-3',
             'capx_-3',
             'naics_-3',
             'ib_-3',
             'ebitda_-3',
             'ebit_-3',
             'nopi_-3',
             'pi_-3',
             'dvt_-3',
             'be_-3',
             'ni_-2',
             'at_-2',
             'epspx_-2',
             'revt_-2',
             'capx_-2',
             'naics_-2',
             'ib_-2',
             'ebitda_-2',
             'ebit_-2',
             'nopi_-2',
             'pi_-2',
             'dvt_-2',
             'be_-2',
             'ni_-1',
             'at_-1',
             'epspx_-1',
             'revt_-1',
             'capx_-1',
             'naics_-1',
             'ib_-1',
             'ebitda_-1',
             'ebit_-1',
             'nopi_-1',
             'pi_-1',
             'dvt_-1',
             'be_-1',
             'ni_0',
             'ni_1',
             'ni_2',
             'ni_3',
             'ni_4',
             'ni_5',
             'at_yoy1',
             'ni_yoy1',
             'epspx_yoy1',
             'revt_yoy1',
             'capx_yoy1',
             'naics_yoy1',
             'ib_yoy1',
             'ebitda_yoy1',
             'ebit_yoy1',
             'nopi_yoy1',
             'pi_yoy1',
             'dvt_yoy1',
             'be_yoy1',
             'at_yoy2',
             'ni_yoy2',
             'epspx_yoy2',
             'revt_yoy2',
             'capx_yoy2',
             'naics_yoy2',
             'ib_yoy2',
             'ebitda_yoy2',
             'ebit_yoy2',
             'nopi_yoy2',
             'pi_yoy2',
             'dvt_yoy2',
             'be_yoy2',
             'at_yoy3',
             'ni_yoy3',
             'epspx_yoy3',
             'revt_yoy3',
             'capx_yoy3',
             'naics_yoy3',
             'ib_yoy3',
             'ebitda_yoy3',
             'ebit_yoy3',
             'nopi_yoy3',
             'pi_yoy3',
             'dvt_yoy3',
             'be_yoy3',
             'at_yoy4',
             'ni_yoy4',
             'epspx_yoy4',
             'revt_yoy4',
             'capx_yoy4',
             'naics_yoy4',
             'ib_yoy4',
             'ebitda_yoy4',
             'ebit_yoy4',
             'nopi_yoy4',
             'pi_yoy4',
             'dvt_yoy4',
             'be_yoy4',
             'at_yoy5',
             'ni_yoy5',
             'epspx_yoy5',
             'revt_yoy5',
             'capx_yoy5',
             'naics_yoy5',
             'ib_yoy5',
             'ebitda_yoy5',
             'ebit_yoy5',
             'nopi_yoy5',
             'pi_yoy5',
             'dvt_yoy5',
             'be_yoy5',
             'CAPEI',
             'bm',
             'evm',
             'pe_op_basic',
             'pe_op_dil',
             'pe_exi',
             'pe_inc',
             'pcf',
             'dpr',
             'npm',
             'opmbd',
             'opmad',
             'gpm',
             'ptpm',
             'cfm',
             'roa',
             'roe',
             'roce',
             'efftax',
             'aftret_eq',
             'aftret_invcapx',
             'aftret_equity',
             'pretret_noa',
             'pretret_earnat',
             'GProf',
             'equity_invcap',
             'debt_invcap',
             'totdebt_invcap',
             'capital_ratio',
             'int_debt',
             'int_totdebt',
             'cash_lt',
             'invt_act',
             'rect_act',
             'debt_at',
             'debt_ebitda',
             'short_debt',
             'curr_debt',
             'lt_debt',
             'profit_lct',
             'ocf_lct',
             'cash_debt',
             'fcf_ocf',
             'lt_ppent',
             'dltt_be',
             'debt_assets',
             'debt_capital',
             'de_ratio',
             'intcov',
             'intcov_ratio',
             'cash_ratio',
             'quick_ratio',
             'curr_ratio',
             'cash_conversion',
             'inv_turn',
             'at_turn',
             'rect_turn',
             'pay_turn',
             'sale_invcap',
             'sale_equity',
             'sale_nwc',
             'rd_sale',
             'adv_sale',
             'staff_sale',
             'accrual',
             'ptb',
             'PEG_trailing',
             'PEG_1yrforward',
             'PEG_ltgforward',
            'med_eps_1',
             'mean_eps_1',
             'std_eps_1',
             'med_sal_1',
             'mean_sal_1',
             'std_sal_1',
             'med_pre_1',
             'mean_pre_1',
             'std_pre_1',
             'med_ebt_1',
             'mean_ebt_1',
             'std_ebt_1',
             'med_eps_2',
             'mean_eps_2',
             'std_eps_2',
             'med_sal_2',
             'mean_sal_2',
             'std_sal_2',
             'med_pre_2',
             'mean_pre_2',
             'std_pre_2',
             'med_ebt_2',
             'mean_ebt_2',
             'std_ebt_2',
             'med_eps_3',
             'mean_eps_3',
             'std_eps_3',
             'med_sal_3',
             'mean_sal_3',
             'std_sal_3',
             'med_pre_3',
             'mean_pre_3',
             'std_pre_3',
             'med_ebt_3',
             'mean_ebt_3',
             'std_ebt_3',
             'med_eps_4',
             'mean_eps_4',
             'std_eps_4',
             'med_sal_4',
             'mean_sal_4',
             'std_sal_4',
             'med_pre_4',
             'mean_pre_4',
             'std_pre_4',
             'med_ebt_4',
             'mean_ebt_4',
             'std_ebt_4',
             'med_eps_5',
             'mean_eps_5',
             'std_eps_5',
             'med_sal_5',
             'mean_sal_5',
             'std_sal_5',
             'med_pre_5',
             'mean_pre_5',
             'std_pre_5',
             'med_ebt_5',
             'mean_ebt_5',
             'std_ebt_5']

In [16]:
mdf3 = mdf2[vars_keep]

In [17]:
mdf3.shape

(692398, 310)

In [18]:
mdf3 = mdf3.drop_duplicates(subset=['jdate', 'permno'])

In [19]:
mdf3.shape

(323254, 310)

In [20]:
mdf3['year'] = mdf3['jdate'].dt.year.astype(int)
mdf3.to_csv(f'{DATA_FOLDER}/lagged_comp-fundr-ibes_for_val_1976-2019.csv')